In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib
# %matplotlib inline
import matplotlib.pyplot as plt
import os
import cv2
from os.path import isfile, join
import glob
import gzip
from PIL import Image
from scipy import misc

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Input,Conv2D,MaxPooling2D,UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import array_to_img

# CNN
import keras
from keras.models import Sequential
from keras.models import load_model
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras import layers, regularizers, optimizers, callbacks
from collections import defaultdict
from keras.optimizers import RMSprop
from keras.layers.normalization import BatchNormalization

# 1. 전처리

In [3]:

#test_iris = []
#test_labels = []


#데이터 불러오기

def create_Trainset(folder):
    iris = []
    labels = []
    
    for file in os.listdir(folder):
        image_path = os.path.join(folder, file)
        image = load_img(image_path, 'rb')
        image = img_to_array(image)
        
        if image.shape[2] == 3:
            image = image.mean(2)

        iris.append(image)  # 이미지 리스트에 저장
        name_index = file.split("_")  # file name: _를 기준으로 split
        name_index = name_index[0]
        name_index = int(name_index[0:3])  # L,R 제외하고 숫자만
        labels.append(name_index)  #이미지에 해당하는 class name 리스트에 저장 
        
    return np.array(iris), np.array(labels)


def create_Testset(folder):
    iris = []
    labels = []
    

    for file in os.listdir(folder):
        image_path = os.path.join(folder, file)
        image = load_img(image_path, 'rb')
        image = img_to_array(image)
        
        if image.shape[2] == 3:
            image = image.mean(2)

        iris.append(image)
        name_index = file.split(".")  # 1.png 에서 숫자만 split
        name_index = int(name_index[0])
        labels.append(name_index)
        
    return np.array(iris), np.array(labels)

# 정규화
def normalization(image):  
    image = image / image.max()
    return image

In [4]:
train_PATH = '/content/drive/MyDrive/iris-training-set'
x_train, y_train = create_Trainset(train_PATH) #불러오기
x_train = normalization(x_train) # 정규화 진행

test_PATH = '/content/drive/MyDrive/iris-test-set'
x_test, _ = create_Testset(test_PATH)
x_test = normalization(x_test)

In [5]:
print('Train images :',x_train.shape)
print('Train labels : ', y_train.shape)
print('Test images : ', x_test.shape)


Train images : (128, 576, 768)
Train labels :  (128,)
Test images :  (256, 576, 768)


## 데이터 불러온 결과,
Train 데이터가 Test 데이터보다 적은 것을 확인할 수 있음

적은 수의 데이터를 학습하는 데에 최적화된 LSTM 모델과 CNN이 결합된 **CNN-LSTM** 모델을 선택하였음

### test 데이터프레임 만들기

In [6]:
test_index = os.listdir(test_PATH)
temp = []

for index in test_index:
    index = index.split(".")
    temp.append(int(index[0]))
    
test_index = temp

# 2. CNN 모델 정의

# 3. 모델 학습
epochs = 30

## 모델 1: 단순 CNN 모델

In [17]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(576, 768)),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(351, activation='softmax')
])


In [18]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary() 

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 442368)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 256)               113246464 
_________________________________________________________________
dense_7 (Dense)              (None, 351)               90207     
Total params: 113,336,671
Trainable params: 113,336,671
Non-trainable params: 0
_________________________________________________________________


In [19]:
model.fit(x_train, y_train, epochs=100)

Epoch 1/100
4/4 [==============================] - 1s 51ms/step - loss: 94.0448 - accuracy: 0.0031
Epoch 2/100
4/4 [==============================] - 0s 45ms/step - loss: 166.2051 - accuracy: 0.0177
Epoch 3/100
4/4 [==============================] - 0s 49ms/step - loss: 123.0512 - accuracy: 0.0281
Epoch 4/100
4/4 [==============================] - 0s 48ms/step - loss: 123.5869 - accuracy: 0.0083
Epoch 5/100
4/4 [==============================] - 0s 47ms/step - loss: 90.2828 - accuracy: 0.0198
Epoch 6/100
4/4 [==============================] - 0s 44ms/step - loss: 61.7189 - accuracy: 0.0198
Epoch 7/100
4/4 [==============================] - 0s 48ms/step - loss: 45.6555 - accuracy: 0.0063
Epoch 8/100
4/4 [==============================] - 0s 46ms/step - loss: 25.2227 - accuracy: 0.0375
Epoch 9/100
4/4 [==============================] - 0s 44ms/step - loss: 6.8928 - accuracy: 0.0115
Epoch 10/100
4/4 [==============================] - 0s 50ms/step - loss: 5.6230 - accuracy: 0.0604
Epoch 11

## 모델 2: CNN-LSTM 모델

In [29]:
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, LSTM, TimeDistributed
from keras.layers import Activation, Dropout


X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size = 0.30, random_state = 36)


In [27]:
num_classes = 64  # 64명의 데이터

model = Sequential()

# Adding CNN Model layers
model.add(TimeDistributed(Conv2D(32, kernel_size = (4 , 2), strides = 1, padding='valid', activation = 'relu', input_shape = (None,576,768,1))))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(Conv2D(64, kernel_size = (4 , 2), strides = 1, padding='valid', activation = 'relu')))
model.add(TimeDistributed(BatchNormalization()))

#model.add(TimeDistributed(Reshape((-1,1))))
model.add(TimeDistributed(MaxPooling2D()))
#model.add(Reshape((1,1)))

# Adding LSTM layers
model.add(LSTM(128, recurrent_dropout=0.2))
model.add(Dropout(rate = 0.2))
model.add(Dense(100))
model.add(Dropout(rate = 0.4))
model.add(Dense(108))
model.add(Dense(num_classes, activation='softmax'))

#model.summary()

ValueError: ignored

In [31]:
num_classes = 64  # 64명의 데이터

model = keras.Sequential([
  layers.Conv2D(64, kernel_size = (2 , 2), strides = 1, padding='valid', 
                                activation = 'relu', input_shape = (None,576,768,1)),
  layers.TimeDistributed(BatchNormalization()),
  layers.Conv2D(128, kernel_size = (2 , 2), strides = 1, padding='valid', activation = 'relu'),
  layers.TimeDistributed(BatchNormalization()),

  layers.MaxPooling2D(),

# Adding LSTM layers
  layers.LSTM(128, recurrent_dropout=0.2),
  layers.Dropout(rate = 0.2),
  layers.Dense(100),
  layers.Dropout(rate=0.4),
  layers.Dense(108),
  layers.Dense(num_classes, activation='softmax')
])

#model.summary()

ValueError: ignored

In [71]:
# Compiling this model
model.compile(loss = 'categorical_crossentropy', 
              optimizer='adam',
              metrics = ['accuracy'])



In [72]:

#Training the Network
history = model.fit(X_train, Y_train, batch_size=32, epochs = 1, validation_data=(X_test, Y_test))
 

ValueError: ignored

# 4. 학습시킨 모델 검증

In [21]:
x_predict = model.predict(x_train)
y_predict = []

print(x_predict.shape)

for image in x_predict:
    y_predict.append(np.argmax(image))

y_predict = np.array(y_predict)

(128, 576, 768)


In [22]:
def accuracy(original, x_predict):
    accuracy = original == x_predict
    accuracy = np.count_nonzero(accuracy)
    
    return accuracy / original.shape[0]
    

In [23]:
accuracy(y_train, y_predict)  #모델의 정확도 출력

0.0

# 4. test set 예측

In [24]:
# test
predictions = model.predict(x_test)  #test데이터의 Y-test 예측하기
print(predictions.shape)


(256, 576, 768)


In [25]:
y_test = []
for image in predictions:
    y_test.append(np.argmax(image))

y_test = np.array(y_test)

#결과
print(y_test)


[ 14152  13505  24274  36710  20936 196211  14153  42606  16338  16338
  36710  36711  37805 201593  19593  13461  37477 193152  26302  13488
  36710  39777 148915 122788  13444  14179 201606 197724  36493  17870
  13426 206952  12769  47423  14896  27836  83064  14948 158134  63932
 204683  16338  13608  13478  12800  36711  20936 195432  36710 216178
  14216  13501  36710  17870 211574  36710 204646  36710 210799  16337
 203113  14913  17870  14220 204659  36710  17870  12788  13467  36710
  16338  36710 201562  13530  15626 205432  17870  80926  13443  19658
 207751  18838  36710  37477  14869  14204  36710  18103  17870  36710
 214628  15617  17870  13601 211597  39777  37477 193605  65142 213124
  50842  17870  54877  60047  12818  35680  12800  16338  73599 207004
  21105  12821  16338 213867  13480  17870  23236  17871  20936  31136
  36710 210799  17870  46241 148915 200800  28785  34969 213883  16338
  12780  22546  13500 112030  17870 197721  17870  17870 215419  18828
  1280

In [ ]:
### csv 저장

test_result_df = pd.DataFrame([x for x in zip(test_index, y_test)], columns=['Image', 'Answer'])
test_result_df.head()

test_result_df = test_result_df.sort_values(by="Image")
test_result_df.head()

test_result_df.to_csv("/content/drive/MyDrive/1971063_김윤서_홍채_1차_답안.csv", index=False)